### Github will not display the folium maps, apparently it blocks the javascript. NBViewer will render the notebook from the github repo, with the maps displayed:
https://nbviewer.jupyter.org/github/rareal/Coursera_Capstone/blob/master/Toronto_neigh_Cluster.ipynb

This is the link to the notebook on github (it's also contained in the path of the link above)
https://github.com/rareal/Coursera_Capstone/blob/master/Toronto_neigh_Cluster.ipynb


# Toronto Neighbourhoods - clustering
#### This is part of the Course [<u><b>Applied Data Science Capstone</b></u>](https://www.coursera.org/learn/applied-data-science-capstone/) on Coursera, to complete the Specialization <u><b>IBM Data Science Professional Certificate</b></u>

This exercise is to cluster the Toronto Neighbourhoods, using the geocodes we got in the second [notebook](https://github.com/rareal/Coursera_Capstone/blob/master/Toronto_Neighborhoods_LatLong.ipynb).

---------------
Importing dependencies:

In [204]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

### 1. Read the dataset generated in the second notebook, with the PotalCodes, Borough, Neighbourhood and the Coordinates 

In [4]:
ToNeighLL = pd.read_csv('Toronto_neigh_latlon.csv',index_col=0)

In [5]:
ToNeighLL.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6555,-79.3626
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7223,-79.4504
4,M7A,Queen's Park,Queen's Park,43.6641,-79.3889


Need to get coordinates for Toronto to plot. I'll use LocationIQ as in the 2nd notebook.

In [6]:
apikey = pd.read_table('locationiq_api_key',header=None).iloc[0,0]

In [7]:
# Search / Forward Geocoding url
search_url = "https://us1.locationiq.com/v1/search.php"
data = {'key': apikey,'q': 'Toronto, Ontario, CA','format': 'json'}
response = requests.get(search_url, params=data)

In [8]:
TO = response.json()[0]
print(TO['display_name'])
print('latitude: ',TO['lat'])
print('longitude: ',TO['lon'])

Toronto, Ontario, M6K 1X9, Canada
latitude:  43.653963
longitude:  -79.387207


### 2. Create a Map for Toronto and plot the Neighbourhoods. 

In [396]:
# Toronto map
toto_map = folium.Map(location=[float(TO['lat']), float(TO['lon'])], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postcode in zip(ToNeighLL['Latitude'], ToNeighLL['Longitude'], 
                                           ToNeighLL['Borough'], ToNeighLL['Neighbourhood'], ToNeighLL['PostalCode']):
    label = '{}, {}, {}'.format(neighborhood, borough, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(  # Circle makes a difined size circle, CircleMarker makes a circle of the same size in all zooms
        [lat, lng],
        radius=200,  # meters
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toto_map)  
    
toto_map

### 3. Gettting nearby venues from foursquare: 

In [183]:
# load foursquare credentials
foursquare_cred = pd.read_csv('foursquare_cred') # saved locally and ignored in .gitignore

CLIENT_ID = foursquare_cred.CLIENT_ID[0] # your Foursquare ID
CLIENT_SECRET = foursquare_cred.CLIENT_SECRET[0] # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Function to parse the result from foursquare (slightly modified from the tutorial, it will include the Borough and the printing will drop the previous line, using `end='\r'`):

In [286]:
def getNearbyVenues(Borough, Neighborhood, latitudes, longitudes, radius=500):
    venues_list=[]
    for Borough, Neighborhood, lat, lng in zip(Borough, Neighborhood, latitudes, longitudes):
        print(Neighborhood,end='\r')
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            Borough, Neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough','Neighborhood','Neighborhood Latitude','Neighborhood Longitude',
                             'Venue','Venue Latitude','Venue Longitude', 'Venue Category']
    return(nearby_venues)

Apply to the Toronto Neighborhoods:

In [287]:
toronto_venues = getNearbyVenues(ToNeighLL.Borough,ToNeighLL.Neighbourhood,ToNeighLL.Latitude,ToNeighLL.Longitude)

Count the number of venues per Neighborhood and Borough:

In [288]:
# Neighborhood venue count
toronto_venues_gr = toronto_venues.groupby(['Neighborhood','Borough']).count()[['Venue']].sort_values(by='Venue')
toronto_venues_gr

,,Venue
Neighborhood,Borough,
"High Park, The Junction South",West Toronto,1
"Newtonbrook, Willowdale",North York,1
"Highland Creek, Rouge Hill, Port Union",Scarborough,1
"Humber Bay Shores, Mimico South, New Toronto",Etobicoke,1
Woburn,Scarborough,1
"Silver Hills, York Mills",North York,2
Scarborough Village,Scarborough,2
"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",Etobicoke,2
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",Scarborough,2


There are a lot of Neighborhoods with very few venues nearby, it will not produce good results if I cluster with those. I'm setting a minimum cutoff of 20 venues:

In [290]:
compare = list(toronto_venues_gr[['Venue']][toronto_venues_gr.Venue>20].reset_index()['Neighborhood'].values)
ref = ToNeighLL.Neighbourhood.values
bool_ind = [x in compare for x in ref]
ToNeighLL_selected = ToNeighLL[bool_ind]

Plotting to see how it looks like:

In [397]:
df=ToNeighLL_selected
# Toronto map
toto_map = folium.Map(location=[float(TO['lat']), float(TO['lon'])], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postcode in zip(df['Latitude'], df['Longitude'], 
                                           df['Borough'], df['Neighbourhood'],
                                            df['PostalCode']):
    label = '{}, {}, {}'.format(neighborhood, borough, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(  # Circle makes a difined size circle, CircleMarker makes a circle of the same size in all zooms
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toto_map)  
toto_map

It's kind of sparse, but otherwise it's to few venues to compare.   
We have to select the Neighborhoods with more than 20 venues:

In [361]:
compare = list(toronto_venues_gr[['Venue']][toronto_venues_gr.Venue>20].reset_index()['Neighborhood'].values)
ref = toronto_venues.Neighborhood.values
bool_ind = [x in compare for x in ref]
toronto_venues_selected = toronto_venues[bool_ind]

In [405]:
toronto_venues_selected.groupby(['Neighborhood','Borough']).count()[['Venue']].sort_values('Venue').head()

,,Venue
Neighborhood,Borough,
"The Beaches West, India Bazaar",East Toronto,23
Davisville,Central Toronto,23
"Harbourfront, Regent Park",Downtown Toronto,24
"Bedford Park, Lawrence Manor East",North York,24
Leaside,East York,27


We end up with:

In [543]:
print(len(toronto_venues_selected.Neighborhood.unique()),'Neighborhoods')

30 Neighborhoods


### 4. Clustering

#### 4.1 Pre-processing:
Now we proceed with the clustering. First is the one hot encoding:

In [414]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_selected[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [535]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_selected['Neighborhood'] 
neigh_ind = list(toronto_onehot.columns).index('Neighborhood') # index of the Neighborhood column
neigh_ind_plus1 = neigh_ind+1

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[neigh_ind]] + list(toronto_onehot.columns[:neigh_ind]) \
                                                    + list(toronto_onehot.columns[neigh_ind_plus1:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
9,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Then we group by Neigborhood and take the mean of the categories, which will be the frequency:

In [536]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000,0.0,0.0,0.040000,0.0,0.010000,0.0,0.000,0.030000,0.0,0.000000,0.000000,0.030000,0.0,0.030000,0.0,0.000000,0.000,0.010000,0.0,0.000000,0.0,0.0,0.02,0.000,0.01,0.020000,0.000,0.0,0.01,0.03,0.01,0.0,0.0,0.000000,0.070000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.010000,0.050000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.020000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.030000,0.01,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,0.0,0.01,0.000,0.01,0.0,0.010000,0.000000,0.020000,0.010000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.040000,0.000000,0.010,0.000000,0.0,0.0,0.0,0.000000,0

30 rows at the end:

In [544]:
toronto_grouped.shape

(30, 227)

#### 4.2 K-means

In [529]:
# number of clusters
k = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=123).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 0, 1, 4, 1, 2, 0, 2, 1, 1, 1,
       1, 2, 2, 2, 2, 4, 3, 1], dtype=int32)

#### 4.3 What distinguishes the clusters?
Printing the 5 most frequent categories per cluster:

In [547]:
df = toronto_grouped.copy()
df2 = pd.concat([df.iloc[:,[0]], pd.DataFrame({'Cluster':kmeans.labels_}), df.iloc[:,1:]],axis=1)
df3 = df2.drop('Neighborhood',1).groupby('Cluster').mean()
df3.reset_index(inplace=True)

print('5 most frequent categories per cluster:')
for x in range(len(df3)):
    print('Cluster {}: {}'.format(x+1,', '.join(list(df3.iloc[[x],:].T.iloc[1:].sort_values(x,ascending=False).index[:5]))))

5 most frequent categories per cluster:
Cluster 1: Clothing Store, Fast Food Restaurant, Coffee Shop, Cosmetics Shop, Women's Store
Cluster 2: Coffee Shop, Café, Restaurant, Bar, Bakery
Cluster 3: Coffee Shop, Restaurant, Café, Italian Restaurant, Sandwich Place
Cluster 4: Greek Restaurant, Ice Cream Shop, Café, Italian Restaurant, Grocery Store
Cluster 5: Pizza Place, Sandwich Place, Fast Food Restaurant, Coffee Shop, Liquor Store


Join cluster labels with Neighborhoods:

In [531]:
to_neigh_clustered = pd.DataFrame()
to_neigh_clustered['Neighborhood'] = toronto_grouped.Neighborhood
to_neigh_clustered['Cluster'] = kmeans.labels_
to_neigh_clustered.head()

,Neighborhood,Cluster
0,"Adelaide, King, Richmond",1
1,"Bedford Park, Lawrence Manor East",2
2,Berczy Park,2
3,"Brockton, Exhibition Place, Parkdale Village",1
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",2


Add Latitude and Longitude:

In [532]:
to_neigh_cl_ll = pd.merge(to_neigh_clustered, ToNeighLL_selected.set_index('Neighbourhood'), 
                          left_on='Neighborhood', right_index=True, how='left', sort=False)
to_neigh_cl_ll.head()

,Neighborhood,Cluster,PostalCode,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",1,M5H,Downtown Toronto,43.6496,-79.3833
1,"Bedford Park, Lawrence Manor East",2,M5M,North York,43.7335,-79.4177
2,Berczy Park,2,M5E,Downtown Toronto,43.6456,-79.3754
3,"Brockton, Exhibition Place, Parkdale Village",1,M6K,West Toronto,43.6383,-79.4301
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",2,M5V,Downtown Toronto,43.6404,-79.3995


#### 4.4 Plotting the clusters:

In [534]:
df=to_neigh_cl_ll
# Toronto map
toto_map = folium.Map(location=[float(TO['lat'])+0.06, float(TO['lon'])+0.01], zoom_start=11)
# set color scheme for the clusters
#k = k
cmap = cm.get_cmap('Set1', k+1)
rainbow = [colors.rgb2hex(cmap(i)[:3]) for i in range(cmap.N)][:-1]

# add markers to map
for lat, lng, cluster, borough, neighborhood in zip(df['Latitude'], df['Longitude'], 
                                           df['Cluster'], df['Borough'], df['Neighborhood']):
    label = 'Cluster: {}, {}, {}'.format(cluster, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(  # Circle makes a difined size circle, CircleMarker makes a circle of the same size in all zooms
        [lat, lng], radius=6, popup=label, color=rainbow[cluster-1],
        fill=True, fill_color=rainbow[cluster-1], fill_opacity=1, parse_html=False).add_to(toto_map)  
toto_map

#### 4.5 Conclusions:
We can see that the Neighborhoods cluster spatially, even though the cluster was made using the Venue categories. Which indicates the similarities between venues in a given region, possibly directed by consumer preference.